In [1]:
import urllib.request
import xarray as xr
import utils.utils
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from urllib.error import HTTPError
from scipy import interpolate
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


file_num = 0

T_RH_PATH = 'D:/ERA5_t_rh_aggregated_3h'
TCW_PATH = 'D:/ERA5_tcw_aggregated_3h'
sample_mswep = xr.open_dataset('D:/sample_mswep.nc').isel(time=0)
box_radius = utils.utils.convert_box_radius(500, res=0.1)

# only do a fraction of the df per notebook, as I run 15 notebooks in parallel to accelerate the process
df = pd.read_csv('data/IBTrACS/ibtracs.above_34.csv')
step = int(np.floor(len(df) / 15)) # 15 is the number of notebooks in parellel I run
start = step * file_num 
end = step * (file_num + 1)
df = df[start:end]


new_df_as_list = []
done = 0
nans_found = 0
timesteps_not_found = 0 # num of files not found because the timestep was not a multiple of 3h
had_inf_bias = 0 # num of files that had infinite bias between ERA5 and MSWEP, likely because the TC was misplaced/absent in ERA5
wrong_shapes = 0 # num of files that had the wrong shape, likely because they were too close to lon=180
was_empty_era5 = 0 # num of files that had no rainfall
was_empty_mswep = 0 # num of files that had no rainfall
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    TC_lat = row.LAT
    TC_lon = row.LON
    if TC_lon > 179.9:
        TC_lon -= 360
    TC_wind = row.WMO_WIND
    idx = row.IDX_ORIG
    iso_time = utils.utils.add_to_iso_time(row.ISO_TIME, hours_delta=1, minutes_delta=30)
    year, month, day, hour = utils.utils.convert_iso_time_3h(iso_time)
    
    with xr.open_dataset(f'{ERA5_PATH}/ERA5_tp_{year}_{month}.nc') as era5_ds:
        # select current timestep in ERA5
        try:
            timestep_slice = utils.utils.slice_ds_timestep(era5_ds, iso_time, 'ERA5')
        except IndexError:
            timesteps_not_found += 1
            continue
        era5_interp_mswep = utils.utils.interp_era5_to_mswep(timestep_slice, sample_mswep)
        # get domain lats and lons (used to crop data around TC centre)
        domain_lats = era5_interp_mswep['latitude'].values
        domain_lons = era5_interp_mswep['longitude'].values
        # find the id inside of domain_lats/domain_lons that corresponds to the TC centre
        TC_lat_id = utils.utils.find_id(TC_lat, domain_lats)
        TC_lon_id = utils.utils.find_id(TC_lon, domain_lons)
        # box ERA5 around IBTrACS TC centre
        boxed_era5 = utils.utils.box_var(era5_interp_mswep, TC_lat_id, TC_lon_id, box_radius, domain_lats, domain_lons, idx)
        if np.isnan(boxed_era5).any():
            boxed_era5 = boxed_era5.fillna(0)
        tp_tot_era5 = np.sum(boxed_era5)*1000
    if tp_tot_era5 < 10:
        was_empty_era5 += 1
        continue
    
    with xr.open_dataset(f'{MSWEP_PATH}/MSWEP_tp_{year}_{month}.nc') as mswep_ds:
        # select current timestep in MSWEP
        try:
            timestep_slice = utils.utils.slice_ds_timestep(mswep_ds, iso_time, 'MSWEP')
        except IndexError:
            timesteps_not_found += 1
            continue
        # get domain lats and lons (used to crop data around TC centre)
        domain_lats = timestep_slice['lat'].values
        domain_lons = timestep_slice['lon'].values
        # find the id inside of domain_lats/lons that corresponds to the TC centre
        TC_lat_id = utils.utils.find_id(TC_lat, domain_lats)
        TC_lon_id = utils.utils.find_id(TC_lon, domain_lons)
        # box ERA5 around IBTrACS TC centre
        boxed_mswep = utils.utils.box_var(timestep_slice, TC_lat_id, TC_lon_id, box_radius, domain_lats, domain_lons, idx)
        tp_tot_mswep = np.sum(boxed_mswep)
    if tp_tot_mswep < 10:
        was_empty_mswep += 1
        continue
    
    # check that the bias (%) between ERA5 and MSWEP is not infinite (it would indicate that
    # the TC in ERA5 was misplaced/missed)
    bias = tp_tot_era5 - tp_tot_mswep
    bias_pc = bias * 100/tp_tot_era5
    bias_is_inf = np.isinf(bias_pc)
    # check that there are no nans in either snapshot
    nans_present = np.isnan(boxed_era5).any() or np.isnan(boxed_mswep).any()
    # check that the shape is correct
    wrong_shape = not(boxed_era5.shape == boxed_mswep.shape == (91, 91))
    if nans_present or bias_is_inf or wrong_shape:
        if wrong_shape:
            wrong_shapes += 1
        elif nans_present:
            nans_found += 1
        elif bias_is_inf:
            had_inf_bias +=1
        continue
    new_df_as_list.append(row)
    # save ERA5 and MSWEP snapshots
    boxed_era5.to_netcdf(f'data/train_large/ERA5_tp/ERA5_tp_cropped_{idx}.nc')
    boxed_mswep.to_netcdf(f'data/train_large/MSWEP_tp/MSWEP_tp_cropped_{idx}.nc')
    # update count of saved files
    done +=1
    
new_df = pd.DataFrame(new_df_as_list, columns = df.columns.to_list())
new_df.to_csv(f'data/IBTrACS/ibtracs.1980-2020.{start}-{end}.csv', index=False)

 61%|██████▏   | 10215/16649 [06:06<03:50, 27.86it/s] 

KeyboardInterrupt



In [2]:
print(f'Skipped because close to 180 lon: {wrong_shapes}')
print(f'Skipped because bias infinite: {had_inf_bias}')
print(f'Skipped because had nans: {nans_found}')
print(f'Skipped because era5 was empty: {was_empty_era5}')
print(f'Skipped because mswep was empty: {was_empty_mswep}')

Skipped because close to 180 lon: 0
Skipped because bias infinite: 0
Skipped because had nans: 0
Skipped because era5 was empty: 0
Skipped because mswep was empty: 0
